## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-11-04-00-00 +0000,wsj,Inside North Korea’s Obsession With Dominating...,https://www.wsj.com/world/asia/north-korea-kim...
1,2026-01-11-03-57-20 +0000,nyt,Trump Is Briefed on Options for Striking Iran ...,https://www.nytimes.com/2026/01/10/us/politics...
2,2026-01-11-03-52-06 +0000,nypost,San Francisco transformed into mock battlezone...,https://nypost.com/2026/01/10/us-news/san-fran...
3,2026-01-11-03-35-47 +0000,nypost,Paramount Skydance now playing the waiting gam...,https://nypost.com/2026/01/10/business/paramou...
4,2026-01-11-03-20-28 +0000,nypost,Disney quietly changes course on controversial...,https://nypost.com/2026/01/10/us-news/disney-q...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2346/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
7,trump,29
88,venezuela,19
45,ice,19
11,iran,16
121,minneapolis,14


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
53,2026-01-10-22-03-34 +0000,nypost,Renee Nicole Good seen ‘dancing’ as she blocks...,https://nypost.com/2026/01/10/us-news/renee-ni...,80
77,2026-01-10-20-08-29 +0000,nyt,"Trump Addresses Venezuela, Greenland and Presi...",https://www.nytimes.com/2026/01/08/us/politics...,70
119,2026-01-10-15-19-51 +0000,nyt,Trump’s $100 Billion Venezuela Oil Plan Gets a...,https://www.nytimes.com/2026/01/09/business/en...,70
132,2026-01-10-13-58-00 +0000,wsj,Trump Presses Oil Executives to Invest in Vene...,https://www.wsj.com/business/energy-oil/trump-...,68
55,2026-01-10-21-44-31 +0000,nypost,"New Jersey’s Petty’s Island, now owned by Vene...",https://nypost.com/2026/01/10/us-news/new-jers...,68


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
53,80,2026-01-10-22-03-34 +0000,nypost,Renee Nicole Good seen ‘dancing’ as she blocks...,https://nypost.com/2026/01/10/us-news/renee-ni...
77,70,2026-01-10-20-08-29 +0000,nyt,"Trump Addresses Venezuela, Greenland and Presi...",https://www.nytimes.com/2026/01/08/us/politics...
128,50,2026-01-10-14-18-09 +0000,nypost,Iran activates ‘missile cities’ as protests ra...,https://nypost.com/2026/01/10/world-news/iran-...
99,35,2026-01-10-17-46-22 +0000,cbc,Rescuers detect 'signs of life' after garbage ...,https://www.cbc.ca/news/world/philippines-garb...
68,33,2026-01-10-20-53-00 +0000,wsj,The Defense Department has selected Owen West ...,https://www.wsj.com/politics/national-security...
87,28,2026-01-10-18-52-27 +0000,nypost,"Ilhan Omar squares off against feds, joins Min...",https://nypost.com/2026/01/10/us-news/ilhan-om...
138,27,2026-01-10-13-00-00 +0000,latimes,One driver accused of two crashes that left th...,https://www.latimes.com/california/story/2026-...
52,27,2026-01-10-22-10-13 +0000,nypost,Thousands pack lefty activist group Monarca’s ...,https://nypost.com/2026/01/10/us-news/minnesot...
102,24,2026-01-10-17-23-27 +0000,nypost,US used powerful mystery weapon that brought V...,https://nypost.com/2026/01/10/world-news/us-us...
75,24,2026-01-10-20-15-48 +0000,nypost,"Israel, Hamas ready to resume war in Gaza as T...",https://nypost.com/2026/01/10/world-news/israe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
